<a href="https://colab.research.google.com/github/omarwadi/EDA/blob/main/California_housing_prices_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#- California housing prices


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [ ]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(housing_prepared, housing_labels)

LinearRegression()

# First try it out on a few instances from the training set:


In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [ ]:
sample_data_prepared = full_pipeline.transform(some_data)

# measure this regression model’s RMSE on the whole training set 
* using Scikit-Learn’s mean_squared_error() function:

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
housing_predictions = linear_model.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print('Mean Squared Error: ', lin_rmse)
print('Average prices of houses: ', housing_labels.mean())

Mean Squared Error:  67593.20745775253
Average prices of houses:  207194.6937378876


# judge on the RMSE result for this model 



- based on the error we got the model isn't very good as the error rate is 34%, and that kind of high, thats almost third of the prdicted values were wrong. 



# Let’s train a Decision Tree Regressor model 
## more powerful model

In [ ]:
from sklearn.tree import DecisionTreeRegressor 

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

# Now let's evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

# judge on the RMSE result for this model 



- its known in machine learing when a model gives perfect results(0% error rate) that means there's something wrong, as it could be overfitting the data, that means if we give the model new data it would give wrong answers as the data is new and unseen from before 

# Evaluation Using Cross-Validation


In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)


display the resultant scores and calculate its Mean and Standard deviation

In [ ]:
print("Scores:", tree_rmse_scores)
print("Mean:", tree_rmse_scores.mean())
print("Standard deviation:", tree_rmse_scores.std())

Scores: [65090.23280193 70638.46937136 68746.98304339 70184.03234903
 72655.69965493 67909.13366532 67374.39198698 68070.66970565
 66448.40410995 70181.46163451]
Mean: 68729.94783230608
Standard deviation: 2113.632148462918


repaet the same steps to compute the same scores for the Linear Regression  model 


In [ ]:
scores = cross_val_score(linear_model, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
linear_rmse_scores = np.sqrt(-scores)
print("Scores:", linear_rmse_scores)
print("Mean:", linear_rmse_scores.mean())
print("Standard deviation:", linear_rmse_scores.std())

Scores: [65000.67382615 70960.56056304 67122.63935124 66089.63153865
 68402.54686442 65266.34735288 65218.78174481 68525.46981754
 72739.87555996 68957.34111906]
Mean: 67828.38677377408
Standard deviation: 2468.0913950652275


## Let’s train one last model the RandomForestRegressor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18498.20746269363

In [ ]:
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
print("Scores:", forest_rmse_scores)
print("Mean:", forest_rmse_scores.mean())
print("Standard deviation:", forest_rmse_scores.std())

Scores: [47061.01124972 51442.24547556 49797.25788321 51903.9639341
 52665.57472294 47046.13628859 47486.12739979 50561.21044463
 49313.06432386 50123.68775669]
Mean: 49740.02794791007
Standard deviation: 1914.689180058856


# Saving every model for later use


In [ ]:
import joblib
joblib.dump(linear_model, "linear_model.pkl")
linear_model_loaded = joblib.load("linear_model.pkl")

joblib.dump(tree_reg, "tree_reg.pkl")
tree_reg_loaded = joblib.load("tree_reg.pkl")

joblib.dump(forest_reg, "forest_reg.pkl")
forest_reg_loaded = joblib.load("forest_reg.pkl")


# Now lets fine-Tune our Model

## 1- Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = [
 {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
 {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
 ]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
  print(np.sqrt(-mean_score), params)

63155.55313139884 {'max_features': 2, 'n_estimators': 3}
55401.073345735545 {'max_features': 2, 'n_estimators': 10}
52652.22210517785 {'max_features': 2, 'n_estimators': 30}
59994.76016957366 {'max_features': 4, 'n_estimators': 3}
53455.27525712462 {'max_features': 4, 'n_estimators': 10}
50123.27187976237 {'max_features': 4, 'n_estimators': 30}
59473.19226289359 {'max_features': 6, 'n_estimators': 3}
52102.270042465374 {'max_features': 6, 'n_estimators': 10}
49829.51247921237 {'max_features': 6, 'n_estimators': 30}
58808.35315857984 {'max_features': 8, 'n_estimators': 3}
51815.390484232026 {'max_features': 8, 'n_estimators': 10}
49712.9492802725 {'max_features': 8, 'n_estimators': 30}
62644.8853231534 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54347.84857616962 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59709.38345194463 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52385.87060970221 {'bootstrap': False, 'max_features': 3, 'n_estimators': 1

# Analyze the Best Models and Their Errors

indicate the relative importance of each attribute

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances


array([6.73255705e-02, 5.85907675e-02, 4.18376887e-02, 1.54422759e-02,
       1.43949987e-02, 1.47154335e-02, 1.35073818e-02, 3.81213143e-01,
       4.08505029e-02, 1.15319377e-01, 6.81900881e-02, 4.73255931e-03,
       1.54699052e-01, 2.03637892e-04, 3.79915811e-03, 5.17836471e-03])

display these importance scores next to their corresponding attribute names:

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.38121314317421684, 'median_income'),
 (0.1546990521141568, 'INLAND'),
 (0.11531937690640857, 'pop_per_hhold'),
 (0.06819008811758702, 'bedrooms_per_room'),
 (0.06732557050958596, 'longitude'),
 (0.05859076753291445, 'latitude'),
 (0.04183768873126078, 'housing_median_age'),
 (0.04085050290079714, 'rooms_per_hhold'),
 (0.015442275890071853, 'total_rooms'),
 (0.014715433539645837, 'population'),
 (0.01439499874049121, 'total_bedrooms'),
 (0.013507381823309532, 'households'),
 (0.005178364713553854, 'NEAR OCEAN'),
 (0.004732559307423016, '<1H OCEAN'),
 (0.0037991581061980317, 'NEAR BAY'),
 (0.0002036378923790769, 'ISLAND')]

## Now is the time to evaluate the final model on the test set.

In [ ]:
final_model = grid_search.best_estimator_
X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()


run full_pipeline to transform the data

In [ ]:
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)


evaluate the final model on the test set

In [ ]:
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) 

# compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

In [ ]:
from scipy import stats

In [ ]:
# CODE HERE
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))

array([47668.51029678, 52072.19945914])